In [0]:
import torch 
import torch.nn as nn
import numpy as np

In [5]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

tcmalloc: large alloc 1073750016 bytes == 0x5861a000 @  0x7f6c529882a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641


In [8]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:

import os
import torch

class Dictionary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = []

    def add_word(self, word):
        if word not in self.word2idx:
            self.idx2word.append(word)
            self.word2idx[word] = len(self.idx2word) - 1
        return self.word2idx[word]

    def __len__(self):
        return len(self.idx2word)


class Corpus(object):
    def __init__(self, path):
        self.dictionary = Dictionary()
        
        # This is very english language specific
        # We will ingest only these characters:
        self.whitelist = [chr(i) for i in range(32, 127)]
        
        self.train = self.tokenize(os.path.join(path, 'train.txt'))
        self.valid = self.tokenize(os.path.join(path, 'valid.txt'))

    def tokenize(self, path):
        """Tokenizes a text file."""
        assert os.path.exists(path)
        # Add words to the dictionary
        with open(path, 'r',  encoding="utf8") as f:
            tokens = 0
            for line in f:
                line = ''.join([c for c in line if c in self.whitelist])
                words = line.split() + ['<eos>']
                tokens += len(words)
                for word in words:
                    self.dictionary.add_word(word)

        # Tokenize file content
        with open(path, 'r',  encoding="utf8") as f:
            ids = torch.LongTensor(tokens)
            #print((ids.size()))
            token = 0
            for line in f:
                line = ''.join([c for c in line if c in self.whitelist])
                words = line.split() + ['<eos>']
                #print(words)
                for word in words:
                    ids[token] = self.dictionary.word2idx[word]
                    token += 1

        return ids

In [0]:
corpus = Corpus('/content/gdrive/My Drive/iris/shakespear/')

In [15]:
#not important
path='/content/gdrive/My Drive/iris/shakespear/'
path=os.path.join(path, 'valid.txt')#concatanates 
print(path)

/content/gdrive/My Drive/iris/shakespear/valid.txt


In [34]:
#not important
whitelist = [chr(i) for i in range(32, 127)]
print(whitelist)

print(type(whitelist))
with open(path, 'r',  encoding="utf8") as f:
            tokens = 0
            for line in f:
                #print(line)
                line = "".join([c for c in line if c in whitelist])
                #print(line)
                words = line.split() + ['<eos>']
                tokens += len(words)
              
with open(path, 'r',  encoding="utf8") as f:
            ids = torch.LongTensor(tokens)
            print(type(ids))
            token = 0
            for line in f:
                line = ''.join([c for c in line if c in whitelist])
                words = line.split() + ['<eos>']
                for word in words:
                    ids[token] = dictionary.word2idx[word]
                    token += 1              


[' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~']
<class 'list'>
<class 'torch.Tensor'>


NameError: ignored

In [0]:
import torch.nn as nn
from torch.autograd import Variable

class RNNModel(nn.Module):

    def __init__(self, vocab_size, embed_size, hidden_size, num_layers, dropout=0.5):
        
        super(RNNModel, self).__init__()
        
        self.encoder = nn.Embedding(vocab_size, embed_size)
        self.drop1 = nn.Dropout(dropout)
        self.drop2 = nn.Dropout(dropout)
        self.rnn = nn.GRU(embed_size, hidden_size, num_layers, dropout=dropout)
        self.decoder = nn.Linear(hidden_size, vocab_size)

        self.init_weights()

        self.hidden_size = hidden_size
        self.num_layers = num_layers

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.fill_(0)
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, input, hidden):
        emb = self.drop1(self.encoder(input))
        output, hidden = self.rnn(emb, hidden)
        output = self.drop2(output)
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded.view(output.size(0), output.size(1), decoded.size(1)), hidden

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        return Variable(weight.new(self.num_layers, batch_size, self.hidden_size).zero_())
    

In [0]:
def batchify(data, batch_size):
    # Work out how cleanly we can divide the dataset into bsz parts.
    nbatch = data.size(0) // batch_size
    print(data.size(0),batch_size, nbatch)
    print("initialy ",data)
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * batch_size)
    print(".narraow() ",data)
    # Evenly divide the data across the bsz batches.
    data = data.view(batch_size, -1)
    print(data)
    data=data.t()
    print("applied a transppose ",data)
    data=data.contiguous()#important just use it
    print(data)
    
    return data

In [13]:
dummy_data = "Once upon a time there was a good king and  a queen"
dummy_data_idx = [corpus.dictionary.word2idx[w] for w in dummy_data.split()]
print(dummy_data_idx)
dummy_tensor = torch.LongTensor(dummy_data_idx) 
op = batchify(dummy_tensor, 2)
for row in op:
    print(row[0],row[1])
    print("%10s %10s" %  (corpus.dictionary.idx2word[row[0]], corpus.dictionary.idx2word[row[1]]))

[9917, 845, 46, 23, 994, 1538, 46, 1171, 2463, 90, 46, 5574]
12 2 6
initialy  tensor([9917,  845,   46,   23,  994, 1538,   46, 1171, 2463,   90,   46, 5574])
.narraow()  tensor([9917,  845,   46,   23,  994, 1538,   46, 1171, 2463,   90,   46, 5574])
tensor([[9917,  845,   46,   23,  994, 1538],
        [  46, 1171, 2463,   90,   46, 5574]])
applied a transppose  tensor([[9917,   46],
        [ 845, 1171],
        [  46, 2463],
        [  23,   90],
        [ 994,   46],
        [1538, 5574]])
tensor([[9917,   46],
        [ 845, 1171],
        [  46, 2463],
        [  23,   90],
        [ 994,   46],
        [1538, 5574]])
tensor(9917) tensor(46)
      Once          a
tensor(845) tensor(1171)
      upon       good
tensor(46) tensor(2463)
         a       king
tensor(23) tensor(90)
      time        and
tensor(994) tensor(46)
     there          a
tensor(1538) tensor(5574)
       was      queen


In [0]:
bs_train = 20       # batch size for training set
bs_valid = 10       # batch size for validation set
bptt_size = 35      # number of times to unroll the graph for back propagation through time
clip = 0.25         # gradient clipping to check exploding gradient

embed_size = 200    # size of the embedding vector
hidden_size = 200   # size of the hidden state in the RNN 
num_layers = 2      # number of RNN layres to use
dropout_pct = 0.5   # %age of neurons to drop out for regularization

In [17]:
train_data = batchify(corpus.train, bs_train)
val_data = batchify(corpus.valid, bs_valid)
vocab_size = len(corpus.dictionary)

1039900 20 51995
initialy  tensor([    0,     1,     2,  ...,     0, 70634,     0])
.narraow()  tensor([    0,     1,     2,  ...,     0, 70634,     0])
tensor([[    0,     1,     2,  ...,    19,  9604,     0],
        [  254,  9605,  9606,  ..., 15550,  1097,    33],
        [ 5415,   202,   183,  ...,  4083,   496, 20229],
        ...,
        [  690,   590,   756,  ...,  1893,   119, 29855],
        [    0,   136,   856,  ...,   756,   705, 11792],
        [ 1045,    33,   111,  ...,     0, 70634,     0]])
applied a transppose  tensor([[    0,   254,  5415,  ...,   690,     0,  1045],
        [    1,  9605,   202,  ...,   590,   136,    33],
        [    2,  9606,   183,  ...,   756,   856,   111],
        ...,
        [   19, 15550,  4083,  ...,  1893,   756,     0],
        [ 9604,  1097,   496,  ...,   119,   705, 70634],
        [    0,    33, 20229,  ..., 29855, 11792,     0]])
tensor([[    0,   254,  5415,  ...,   690,     0,  1045],
        [    1,  9605,   202,  ...,   590, 

In [0]:
model = RNNModel(vocab_size, embed_size, hidden_size, num_layers, dropout_pct)
criterion = nn.CrossEntropyLoss()
def get_batch(source, i, evaluation=False):
    seq_len = min(bptt_size, len(source) - 1 - i)
    data = Variable(source[i:i+seq_len], volatile=evaluation)
    data.contiguous()
    target = Variable(source[i+1:i+1+seq_len].view(-1))
    #cuda 
    return data, target

In [0]:
data, target = get_batch(train_data, 1)

In [0]:
import torch.optim as optim

def train(data_source, lr):
    
    
    model.train()
    total_loss = 0
    hidden = model.init_hidden(bs_train)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    for batch, i in enumerate(range(0, data_source.size(0) - 1, bptt_size)):
        
        data, targets = get_batch(data_source, i)

        
        hidden = Variable(hidden.data)
        
        #if cuda.is_available():
        #    hidden = hidden.cuda()
        
        
        optimizer.zero_grad()
        
        output, hidden = model(data, hidden)
        loss = criterion(output.view(-1, vocab_size), targets)
        loss.backward()

        
        torch.nn.utils.clip_grad_norm(model.parameters(), clip)
        
        optimizer.step()
        total_loss += len(data) * loss.data
        
    return total_loss[0] / len(data_source)

In [0]:

def evaluate(data_source):
    # Turn on evaluation mode which disables dropout.
    model.eval()
    total_loss = 0
    hidden = model.init_hidden(bs_valid)
    
    for i in range(0, data_source.size(0) - 1, bptt_size):
        data, targets = get_batch(data_source, i, evaluation=True)
        
        #if cuda.is_available():
        #   hidden = hidden.cuda()
            
        output, hidden = model(data, hidden)
        output_flat = output.view(-1, vocab_size)
        
        total_loss += len(data) * criterion(output_flat, targets).data
        hidden = Variable(hidden.data)
        
    return total_loss[0] / len(data_source)

In [0]:
best_val_loss = None

In [0]:
def run(epochs, lr):
    global best_val_loss
    
    for epoch in range(0, epochs):
        train_loss = train(train_data, lr)
        val_loss = evaluate(val_data)
        print("Train Loss: ", train_loss, "Valid Loss: ", val_loss)

        if not best_val_loss or val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), "./4.model.pth")


In [0]:
run(5, 0.001)


In [0]:
run(5, 0.001)

In [0]:
num_words = 200
temperature = 1
model = RNNModel(vocab_size, embed_size, hidden_size, num_layers, dropout_pct)
model.load_state_dict(torch.load("./4.model.pth"))

#if cuda.is_available():
#    model.cuda()
    
model.eval()

In [0]:
temperature = 0.8

In [0]:
hidden = model.init_hidden(1)
idx = corpus.dictionary.word2idx['I']
input = Variable(torch.LongTensor([[idx]]).long(), volatile=True)

if cuda.is_available():
    input.data = input.data.cuda()

print(corpus.dictionary.idx2word[idx], '', end='')

for i in range(num_words):
    output, hidden = model(input, hidden)
    word_weights = output.squeeze().data.div(temperature).exp().cpu()
    word_idx = torch.multinomial(word_weights, 1)[0]
    input.data.fill_(word_idx)
    word = corpus.dictionary.idx2word[word_idx]

    if word == '<eos>':
        print('')
    else:
        print(word + ' ', end='')